# Изменение порядка слов в предложении при переводе (или до перевода)


При переводе, как правило, синтаксическая структура предложения меняется (т.е. перевод не только отображает "английские" слова в "русские", но и меняет их местами; некоторые слова могут исчезнуть или, наоборот, появиться).

Меня интересует разница синтаксиса между различными славянскими языками -- их синтаксис достаточно близок друг к другу, но при этом существует множество нюансов (как описанных в литературе, так и нет).



## Мотивация

Моё хобби -- это разработка NLP-инструментов для межславянского искусственного языка. Межславянскы језык јест језык, разумливы приблизно всим словјанам без ученја (грубо говоря, можно назвать его "славянским Эсперанто").

В основном он добивается своей цели посредством тщательно курированного словосбора, но также важными являются грамматика и фонетика. Нажалость, спецификации синтаксиса для него практически не существует: учебники ограничиваются фразами типа "порядок слов обычно Subject Verb Object", но на самом деле очень многое остаётся недосказанным. Синтаксис (т.е. порядок слов) влияет на понятность и на степень того, насколько скрипят мозги у читателя/слушателя.

Этот проект посвящён некоторым аспектам славянских языков с прицелом на то, как адаптировать синтаксис данного природного языка для того, чтобы увеличить его разумливость. Желательно, автоматически.

Я попробую изучить эти явления при помощи модели `facebook/nllb-200-distilled-600M`.

## Вопрос 1: обнуление субъекта и копулы

Воспроизводит ли NLLB-200 некоторые известные синтаксические явления?

К числу наиболее частых явлений относится pro-drop (пропуск личных местоимений) и zero copula (опускание копулы, т.е. глагола-связки "быть").

Эти аспекты ~~суть~~ проблематичны в контексте межславянского языка, потому что разные языки используют разные стратегии пропускания и это может привести к проблемам в понятности. Например, сравним пары чешских и русских предложений:

> Мы ~~есть~~ довольны 
> 
> ~~My~~ Jsme spokojeni;  
> 
> Я сегодня ~~есть~~ в школе 
> 
> ~~Já~~ Jsem dnes ve škole.
> 
> А ты куда ~~идёшь~~? 
> 
> Kam ~~ty~~ jdeš?
> 
> Вы когда ~~отправитесь~~ обратно? 
> 
> Kdy ~~vy~~ se vracíte?

или следующее польское предложение:

> Wiedziałem, że do nas wrócisz
> 
> знал_есмь, что к нам вернёшь
> 
> "~~Я~~ знал, что ~~ты~~ к нам вернешься"

Видно, что чешская стратегия построения коротких предложений (пропуск местоимений) в каком-то смысле является противоположностью русской стратегии пропуска глаголов.

Также в западнославянских языках можно опускать местоименное подлежащее, относящееся к только что упомянутому субъекту. Обе эти фразы на польском являются допустимыми:

> Piotr spotkał Ewę. Ona powiedziała mu, że...
> 
> Piotr spotkał Ewę. Powiedziała mu, że...

В то же время в русском языке пропуск местоимения "она" практически немыслим.

> Пётр встретил Наташу. Она рассказала ему, что...


### пропуск местоимений

Если говорить о природных славянских языках, то вопрос пропуска местоимений/предикатов можно назвать умозрительным. Однако в контексте межславянского этот вопрос ~~есть~~ действительно важный. ~~Я~~ приведу два реально встретившихся мне примера:

Говоритель -- хорват.
> Odlično, ja už jesm govoril s jim, ale ničto ne jest govoril
>
> "Отлично, я уже говорил с ним, но ~~он~~ ничего не ответил"
>
> "Odlično, pitao sam ga, ali ~~on~~ nije odgovorio"

По правилам хорватского синтаксиса (как и межславянского) личное местоимение "он" здесь ~~есть~~ избыточно, поскольку нужная информация здесь передаётся вспомогательным глаголом `jest` (который изменяется по числам и лицам: ja jesm, ty jesi, on jest; можно провести аналогию с русским "я иду", которое без потери информации можно сократить до просто "иду"). Тем не менее, это вызывает проблемы с пониманием из-за своей непривычности.

(также интересно сравнить с "я уже с ней говорил, но ничего не рассказала")

Зеркальный пример, где говоритель -- русскоговорящий.

> Odpisal tobě privatno
>
> "~~Я~~ ответил тебе приватно"
>
> "~~Ja~~ odgovorio sam ti privatno"

Фразы похожей конструкции часто встречаются в русском (и других восточнославянских языках): например, "читал книгу" или "смотрел вчера интересный фильм". Строго говоря, здесь уже грамматически невозможно восстановить пропущеного агента, коим может быть как "я", так и "он". Благодаря сложившейся традиции все русскоговорящие обычно понимают, что здесь подразумевается "я". 

Однако для западных и южных славян это в лучшем случае непонятно, в худшем вообще неграмматично. Я спросил нескольких южных славян по поводу этой фразы и пришёл к выводу, что без дополнительного контекста южане скорее подумают, что здесь пропущено местоимение "он".

### пропуск копулы

Это явление характерно для русского языка (а также, в чуть меньшей степени, для украинского и беларусского). Глагол-связка "быть" исчезает в настоящем времени, хотя существует в прошедшем и будущем:

> Мне это было интересно. Он был студентом. ("он был студент" тоже допустимо)
> 
> Мне это ~~есть~~ интересно. Он студент.
> 
> Мне это будет интересно. Он будет студентом. ("он будет студент" тоже допустимо)

Это затрудняет восприятие русского текста западными и южными славянами.

### глаголы первого и третьего лица настоящего времени

Дополнительную проблему вызывают глаголы в формах первого и третьего лица. Формально говоря, грамматика межславянского однозначно указывает на нужное местоимение:

> Ja dělaju / ja dělam
> 
> My dělajemo / my dělamo
> 
> Oni delajut

Однако на практике опускать местоимения не рекомендуется, поскольку снаружи межславянского однозначность разрушается. Примерно в половине славянских языков глаголы выглядят примерно как "я делаю, мы делаем, они делают", а в половине "я делам, мы деламо, они делаю".

Таким образом, местоимение требуется даже в таких очевидно однозначных с точки зрения русского языка случаях как "иду, вижу, слышу, читаю".

In [77]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", output_attentions=True, return_dict_in_generate=True)

In [7]:
from razdel import sentenize

def translate(text, src_lang, tgt_lang, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    results = []
    for sent in sentenize(text):
        inputs = tokenizer(sent.text, return_tensors='pt')
        result = model.generate(
            **inputs.to(model.device), 
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
            **kwargs
        )
        results.append(
            tokenizer.decode(result[0], skip_special_tokens=True)
        )
    return results


NLLB200 позволяет переводить тексты с любого славянского языка на любой славянский без посредника в виде pivot language (например, английского).

In [341]:


SLAVIC_LANGS = [
    'bel', 'ukr', 'rue', 'rus', 'bos', 'hrv', 'srp', 'cnr', 'slv', 'bul', 'mkd', 'chu', 'wen', 'dsb', 'hsb', 'pol', 'szl', 'csb', 'pox', 'ces', 'czk', 'slk',
]


ietf_bcp = {
    'bel': "bel_Cyrl",
    'ukr': "ukr_Cyrl",
    'rue': None,
    'rus': "rus_Cyrl",
    'bos': "bos_Latn",
    'hrv': "hrv_Latn",
    'srp': "srp_Cyrl", 
    'cnr': None,
    'slv': "slv_Latn",
    'bul': "bul_Cyrl",
    'mkd': "mkd_Cyrl",
    'chu': None,
    'wen': None,
    'dsb': None,
    'hsb': None,
    'pol': "pol_Latn",
    'szl': None,
    'csb': None,
    'pox': None,
    'ces': "ces_Latn",
    'czk': None,
    'slk': "slk_Latn",
}

translation = {}
for LANG in SLAVIC_LANGS:
    l = ietf_bcp[LANG]
    if l is None:
        continue
    print(LANG)
    translation, _ = translate("kanapka z łososiem", "pol_Latn", l)
    print(translation)
    translation, _ = translate("медведь со мной", "rus_Cyrl", l)
    print(translation)
    translation, _ = translate("bear with me", "eng_Latn", l)
    print(translation)


bel
['сэндвич з сальсамі']
['Медведзь са мной']
['выцярпеце са мной']
ukr
['сэндвич з лососями']
['Медведь зі мною']
['Витримай мене']
rus
['лосось-санск']
['медведь со мной']
['Будь со мной терпелив.']
bos
['sendvič sa zalosom']
['Medved sa mnom.']
['-Budi sa mnom.']
hrv
['Sjednica sa zalosom']
['Medvjed sa mnom.']
['-Budi strpljivi sa mnom.']
srp
['суствер са лососом']
['Медвеђа са мном']
['Држи ме са мном.']
slv
['sendvič z lasom']
['Medved z mano.']
['-Straj z mano.']
bul
['сандвич с лососи']
['Мечката е с мен.']
['- Не ме мърдай.']
mkd
['сандвич со лососи']
['Мечката со мене.']
['Држи се со мене.']
pol
['kanapki z łosośą']
['niedźwiedź ze mną']
['- Nie.']
ces
['sendvič s lososem']
['medvěd se mnou.']
['- Nechte mě být.']
slk
['sendvič s lososom']
['medveď so mnou']
['- Nechaj ma.']


По фрагменту "лосось-санск" видно, что модель (тем более, квантизованная и дистилированная) работает неидеально :)

Тем не менее, это даёт нам возможность заметить ещё одну особенность NLLB200: возможность переводить с поломатого русского на нормальный русский.

In [359]:
translation, _ = translate("я узрел блюдо 'сендвитч с лососем' внутри меню", "rus_Cyrl", "rus_Cyrl")

translation

['Я видел в меню бутерброд с лососом.']

### вопрос о методологии

Этот раздел вдохновлён работой https://www.dialog-21.ru/media/1335/124.pdf (2010 год), где автор анализирует параллельный чешско-русский корпус и описывает, какими именно различиями в синтаксе обусловлены наиболее частые различия порядка слов.

Методика определения различий весьма груба: оба параллельных предложения проходят через PoS-tagging (используя чешский и русский анализаторы соответственно), далее между полученными последовательностями тэгов вычисляется расстояние Левенштейна (например, `NOUN VERB ADJ NOUN` и `NOUN AUX VERB ADJ NOUN` находятся на расстоянии 1 друг от друга). Тем не менее, этот подход оказывается достаточным для выявления многих интересных закономерностей, а выводы автора кажутся мне интересными.

Интересно, как соотносятся выводы 2010 года и технологии 2024 года? Иными словами, если бы автор статьи сравнивала не пары ручных переводов, а предложение и его автоперевод? Получилось ли бы у неё найти те же самые закономерности и сделать такие же выводы?


Попробуем воспроизвести пример с pro-drop.

In [342]:
for LANG in SLAVIC_LANGS:
    l = ietf_bcp[LANG]
    if l is None:
        continue
    print(LANG)
    translation, _ = translate("Peter met Eve and she told him about the lost necklace.", "eng_Latn", l)
    print(translation)


bel
['Пётр сустрэў Эву, і яна расказала яму пра страчанае шаліца.']
ukr
['Пітер зустрів Еву, і вона розповіла йому про втрачений ожерелок.']
rus
['Питер встретил Еву, и она рассказала ему о потерянной ожерелье.']
bos
['Peter je upoznao Eve i ona mu je rekla za izgubljen ogrlicu.']
hrv
['Peter je upoznao Eve i ona mu je rekla za izgubljen ogrlicu.']
srp
['Питер је упознао Еву и она му је рекла о изгубљеном огрлицу.']
slv
['Peter je srečal Eve in mu je povedala o izgubljeni ogrlice.']
bul
['Питър се срещна с Ева и тя му каза за изгубената огърлица.']
mkd
['Питер ја запозна Ева и таа му кажа за изгубеното огрлица.']
pol
['Peter poznał Ewę i ona opowiedziała mu o utraconym naszyjniku.']
ces
['Peter se setkal s Eve a ona mu řekla o ztraceném náhrdelníku.']
slk
['Peter sa stretol s Eve a ona mu povedala o stratenej náhrdelnici.']


In [343]:
for LANG in SLAVIC_LANGS:
    l = ietf_bcp[LANG]
    if l is None:
        continue
    print(LANG)
    translation, _ = translate("Piotr spotkał Ewę i powiedziała mu o utraconym naszyjniku.", "pol_Latn", l)
    print(translation)


bel
['Пётар сустрэў Эву і распавёў яму пра страчанае шыі.']
ukr
['Петро знайшов Єву і розповіла йому про втрачену шейку.']
rus
['Петр встретил Еву и рассказал ему о потерянной шерсти.']
bos
['Petar je upoznao Evu i rekla mu da je izgubila vratu.']
hrv
['Petar je upoznao Evu i ispričao mu o izgubljenom vratu.']
srp
['Петр је срео Еву и рекао му о изгубљеној врећи.']
slv
['Peter je srečal Evo in mu povedal, da je izgubila vratu.']
bul
['Петър се срещнал с Ева и му казал за изгубената му шиша.']
mkd
['Петар ја сретнал Ева и му кажала за изгубеното јаче.']
pol
['Peter spotkał Ewę i opowiedziała mu o utraconym sznurze.']
ces
['Petr potkal Evu a řekl mu o ztraceném náhrdelníku.']
slk
['Peter sa stretol s Evou a povedal mu o stratenej náhrdelnici.']


In [344]:
for LANG in SLAVIC_LANGS:
    l = ietf_bcp[LANG]
    if l is None:
        continue
    print(LANG)
    translation, _ = translate("'The wind is too strong', he said", "eng_Latn", l)
    print(translation)
    translation, _ = translate("'The wind is too strong', she said", "eng_Latn", l)
    print(translation)


bel
['"Вятр занадта моцны", сказаў ён']
['"Вятр занадта моцны", сказала яна']
ukr
['"Вятр занадто сильний", сказав він.']
['"Вятр занадто сильний", сказала вона']
rus
['"Ветр слишком сильный", сказал он.']
['"Ветр слишком сильный", сказала она.']
bos
['"Vjetar je previše jak", rekao je on.']
['"Vjetar je previše jak", rekla je ona']
hrv
['"Vjetar je previše jak", rekao je']
['"Vjetar je previše jak", rekla je ona']
srp
['"Вятр је превише јак", рекао је']
['"Вятр је превише јак", рекла је']
slv
['"Veter je preveč močan", je rekel']
['"Veter je preveč močan", je rekla']
bul
['"Вятърът е твърде силен", каза той.']
['"Вятърът е твърде силен", каза тя.']
mkd
['"Ветрот е премногу силен", рече тој']
['"Ветрот е премногу силен", рече таа']
pol
['"Wiatr jest zbyt silny", powiedział']
['"Wiatr jest zbyt silny", powiedziała']
ces
['"Vítr je příliš silný", řekl.']
['"Větr je příliš silný", řekla.']
slk
['"V vetre je príliš silný", povedal']
['"V vetre je príliš silné", povedala']


С третьего раза частично получилось.

In [345]:
for LANG in SLAVIC_LANGS:
    l = ietf_bcp[LANG]
    if l is None:
        continue
    print(LANG)
    translation, _ = translate("I asked him, but he did not respond", "eng_Latn", l)
    print(translation)


bel
['Я спытаў яго, але ён не адказаў']
ukr
['Я запитав його, але він не відповів.']
rus
['Я спросил его, но он не ответил.']
bos
['Pitao sam ga, ali nije odgovorio.']
hrv
['Pitao sam ga, ali nije odgovorio.']
srp
['Питала сам га, али није одговорио.']
slv
['Vprašala sem ga, a ni odgovoril.']
bul
['Попитах го, но той не отговори.']
mkd
['Го прашав, но не одговори.']
pol
['Spytałem go, ale nie odpowiedział.']
ces
['Zeptal jsem se ho, ale neodpověděl.']
slk
['Spýtal som sa ho, ale neodpovedal.']


Здесь тоже заметна тенденция к pro-drop, хоть и не совсем последовательная.

In [346]:
for LANG in SLAVIC_LANGS:
    l = ietf_bcp[LANG]
    if l is None:
        continue
    print(LANG)
    translation, _ = translate("I've seen an interesting movie yesterday", "eng_Latn", l)
    print(translation)
    translation, _ = translate("Видел вчера интересный фильм.", "rus_Cyrl", l)
    print(translation)


bel
['Я бачыў учора цікавы фільм']
['Я бачыў цікавы фільм учора.']
ukr
['Я бачив вчора цікавий фільм.']
['Вчора я бачив цікавий фільм.']
rus
['Я вчера посмотрел интересный фильм.']
['Я вчера посмотрел интересный фильм.']
bos
['Jučer sam vidio zanimljiv film.']
['Vidio sam zanimljiv film juče.']
hrv
['Vidio sam zanimljiv film jučer.']
['Vidio sam zanimljiv film jučer.']
srp
['Учерашњи филм сам видео.']
['Идио сам интересантни филм вчера.']
slv
['Včeraj sem videl zanimivo film.']
['Včeraj sem videl zanimivo film.']
bul
['Вчера видях интересен филм.']
['Вчера видях интересен филм.']
mkd
['Вчера видов интересен филм.']
['Вчера видов еден интересен филм.']
pol
['Widziałem wczoraj ciekawy film.']
['Widziałem wczoraj ciekawy film.']
ces
['Včera jsem viděl zajímavý film.']
['Včera jsem viděl zajímavý film.']
slk
['Včera som videl zaujímavý film.']
['Včera som videl zaujímavý film.']


Удивительно, но NLLB200 понял, что в русской фразе пропущено именно "я", а не "он".

Попробуем перевести с русского на чешский фразы, в которых пропущена копула (но это не очевидно).

In [241]:
r, outputs = translate("Писать статьи тяжело. Сейчас холодно. На улице светло.", "rus_Cyrl", "ces_Latn", output_attentions=True, return_dict_in_generate=True)

r

['Psaní článků je těžké.', 'Je zima.', 'Na ulici je světlo.']

In [242]:
r, outputs = translate("Writing articles is difficult. It's cold. It's bright outside", "eng_Latn", "ces_Latn", output_attentions=True, return_dict_in_generate=True)

r

['Psaní článků je těžké.', 'Je zima.', 'Venku je světlo.']

In [250]:
r, outputs = translate(
    "Мы знаем, какие действия в наших интересах. Мы знаем, какие действия суть в наших интересах. Знаю, что именно в моих интересах. Я знаю, что именно в моих интересах. Это мне любопытно.", 
    "rus_Cyrl", "ces_Latn", 
    output_attentions=True, return_dict_in_generate=True)

r

['Víme, co je v našem zájmu.',
 'Víme, co je v našem zájmu.',
 'Vím, že je to pro mě nejlepší.',
 'Vím, co je pro mě nejlepší.',
 'To mě zajímá.']

In [251]:
r, outputs = translate(
    "Мы знаем, какие действия в наших интересах. Мы знаем, какие действия суть в наших интересах. Знаю, что именно в моих интересах. Я знаю, что именно в моих интересах. Это мне любопытно.", 
    "rus_Cyrl", "pol_Latn", 
    output_attentions=True, return_dict_in_generate=True)

r

['Wiemy, co jest dla nas najlepsze.',
 'Wiemy, co jest dla nas najlepsze.',
 'Wiem, że to jest dla mnie najlepsze.',
 'Wiem, co jest dla mnie najlepsze.',
 'To ciekawe.']

**Заключение**: хотя по качеству перевода модель явно уступает человеческому (и даже некоторым более специализированным моделям, заточенным под "крупные" языки -- например, https://translate.yandex.ru/), на основе таких экспериментов действительно можно сделать более-менее правильные выводы о различиях в синтаксическом строе разных языков.

## Вопрос 2: Починка предложений

Продолжая разбор упомянутой выше статьи -- можем ли мы при помощи NLLB200 и других технологий восстановить опущенную в русском языке копулу?

Основной датасет взят из этой же статьи (доступен по адресу https://ufal.mff.cuni.cz/umc/cer/). После того, как все PoS-тэги получены, можно найти примеры предложений с опущеными местоимениями и/или копулой: если в русском предложении нет глаголов, а чешском они есть, то скорее всего дело в zero copula.


In [255]:
import spacy_udpipe

spacy_udpipe.download("cs") # download English model
spacy_udpipe.download("ru") # download English model

nlp_cs = spacy_udpipe.load("cs")
nlp_ru = spacy_udpipe.load("ru")


Already downloaded a model for the 'cs' language
Already downloaded a model for the 'ru' language


In [ ]:
ru_sents = []
cs_sents = []
with open(r"umc-0.1-corpus\Czech-Russian.1-1.txt", "r", encoding='utf8') as f:
    for line in f:
        cs_text, ru_text = line.split("\t")
        ru_sents.append(ru_text)
        cs_sents.append(cs_text)


In [ ]:
from collections import Counter
token_pos_seq = {"ru": [], "cs": []}


N = 0
total = 0

with open(r"umc-0.1-corpus\Czech-Russian.1-1.txt", "r", encoding='utf8') as f:
    for line in f:
        total += 1
        # print(line.split("\t"))
        cs_text, ru_text = line.split("\t")
        doc_cs = nlp_cs(cs_text)
        doc_ru = nlp_ru(ru_text)
        token_pos_seq["cs"].append([token.pos_ for token in doc_cs])
        token_pos_seq["ru"].append([token.pos_ for token in doc_ru])
        
        if token_pos_seq["cs"][-1] == token_pos_seq["ru"][-1]:
            N += 1

cnt = Counter()

for lang in ['cs', 'ru']:
    for seq in token_pos_seq[lang]:
        cnt.update(seq)



In [ ]:
zero_copula_list = []
prodrop_list

for i, (ru_seq, cs_seq) in enumerate(zip(token_pos_seq['ru'], token_pos_seq['cs'])):
    cz_has_copula = ("VERB" in cs_seq) or ("AUX" in cs_seq)
    ru_has_copula = ("VERB" in ru_seq) or ("AUX" in ru_seq)
    if cz_has_copula and not ru_has_copula:
        zero_copula_list.append((ru_seq, cs_seq, ru_sents[i], cs_sents[i]))
    cz_has_pronoun = ("PRON" in cs_seq)
    ru_has_pronoun = ("PRON" in ru_seq)
    if ru_has_pronoun and not cz_has_pronoun:
        prodrop_list.append((ru_seq, cs_seq, ru_sents[i], cs_sents[i]))        


In [ ]:
import pickle

with open("zero_copula_list.pkl", "wb") as f:
    pickle.dump(zero_copula_list, f)

with open("prodrop_list.pkl", "wb") as f:
    pickle.dump(prodrop_list, f)



In [254]:
import pickle

with open("zero_copula_list.pkl", "rb") as f:
    zero_copula_list = pickle.load(f)

with open("prodrop_list.pkl", "rb") as f:
    prodrop_list = pickle.load(f)



In [360]:
len(zero_copula_list), len(prodrop_list)

(4573, 17962)

### Стратегии возврата копулы

Первая стратегия хорошо работает с предложениями вида "X - это Y":
1) втыкиваем слово "являться" вместо дефиса

2) запрещаем декодеру использовать токен "-"

In [386]:
bad_words_ids = [tokenizer.encode(word, add_special_tokens=False) for word in "-"]

In [283]:
r, outputs = translate(
    "Германия , Франция и Италия - богатые страны .".replace(" - ", "являться"), 
    "rus_Cyrl", "rus_Cyrl", 
    output_attentions=True, return_dict_in_generate=True,
    bad_words_ids = bad_words_ids
)
r

['Германия , Франция и Италия являются богатыми странами .']

Вторая стратегия:

1) просто вставляем глагол-связку в конец предложения

2) Какое должно быть время для глагола-связки? Перебираем несколько основных вариантов: "являться", "является", "было". В дальнейшем можно несложным образом заменить "был" на "есть", чтобы вернуть предложение в настоящее время.

In [389]:
for entry in zero_copula_list[100:111]:
    ru_text, cs_text = entry[2].strip(), entry[3].strip()
    ru_text = ru_text.replace(' – ', ' - ').replace(' : ', ' - ')
    if " - " in ru_text:
        continue
    print(ru_text)
    variants = set()

    for sent_add_cand in "являться является был было была были".split(): 
            # for replacement in ["", " - "]:
            res, _ = translate(
                ru_text[:-1] + " " + sent_add_cand + ".",
                "rus_Cyrl", "rus_Cyrl",
                output_attentions=0, return_dict_in_generate=1)
            variants.add(res[0])
    print(list(variants))
    print()
    print()


На улицах необузданное распространение пессимизма .
['На улицах было бесконечное распространение пессимизма.', 'На улицах бесконечное распространение пессимизма.', 'На улицах необузданное распространение пессимизма.']


У Европы впереди два пути .
['Европа имеет два пути.', 'У Европы есть два пути.', 'У Европы были два пути.']


Жозе Бове против бедных
['Джозе Бове была против бедных.', 'Джозе Боуэ был против бедных.', 'Джозе Боу был против бедных.', 'Джозе Бове против бедных.', 'Джозе Боу против бедняков.']


Эта история почти полностью неправдоподобна .
['Эта история была почти полностью неправдоподобной.', 'Эта история почти полностью неправда.']


Все остальное неправда .
['Все остальное неправда.', 'Все остальное было неправдой.']


Что же в этом привлекательного ?
['Что было привлекательно в этом?', 'Что в этом привлекательно?', 'Что же привлекательно в этом?']




Третья стратегия требует бОльших ресурсов:

1) делаем разбор предложения через udpipe

2) проверяем, есть ли там токены со связями типа `cop`, `nsubj` или `csubj`

3) вставляем глагол-связку вместо `cop` или перед `nsubj` или `csubj`. Какое должно быть время для глагола-связки? Перебираем четыре основных варианта

In [392]:

for ru_text in [
    "Сколько платить , еще не решено .",
    "Забыть эту поездку трудно.",
    "Нужны компетентные профессионалы .",
    "На улицах необузданное распространение пессимизма"
]:
    print(ru_text)
    doc = nlp_ru(ru_text)
    variants = set()
    
    for token in doc:
        if token.dep_ in ['cop', 'nsubj', 'csubj', 'ROOT']:
            if token.dep_ == 'cop':
                src_text = token.text
                ru_text_cand = ru_text
            else:
                ru_text_cand = ru_text.replace(token.text, f"<subj> {token.text}")
                src_text = "<subj>"
            # for replacement_cand in ["будет", "было", "являться", "является"]:
            # for replacement_cand in ["являться", "является", "is"]:
            for replacement_cand in ["является", "будет", "было", "являться"]:
                ru_text_cand2 = ru_text_cand.replace(src_text, replacement_cand)
                # print(ru_text_cand2)
                res, _ = translate(
                    ru_text_cand2,
                    "rus_Cyrl", "rus_Cyrl",
                    output_attentions=0, return_dict_in_generate=1,
                    bad_words_ids=bad_words_ids
                )
                variants.add(res[0])
    print(list(variants))
    print()


Сколько платить , еще не решено .
['Сколько придется заплатить , пока не решено .', 'Пока не решено , сколько заплатить .', 'Сколько будет платить , пока не решено .', 'Сколько заплатить , пока не решено .', 'Сколько было заплачено , еще не решено .', 'Сколько стоит заплатить , пока не решено .', 'Сколько будет платить , еще не решено .']

Забыть эту поездку трудно.
['Это было трудно забыть.', 'Это трудно забыть.', 'О том путешествии трудно забыть.', 'Очень трудно забыть о поездке.', 'Это будет трудно забыть.']

Нужны компетентные профессионалы .
['Нужны будут компетентные профессионалы .', 'Нужны были компетентные профессионалы .', 'необходимы компетентные профессионалы .', 'Должны быть компетентные профессионалы .', 'Нужны компетентные профессионалы .', 'нужны были компетентные профессионалы .']

На улицах необузданное распространение пессимизма
['На улицах будет бескорыстно распространяться пессимизм', 'На улицах бескорыстно распространялся пессимизм', 'На улицах есть бесконечный ра

Увы, незаметно, чтобы эта более сложная стратегия давала здесь какой-то заметный выигрыш (возможно даже наоборот).

Также нужно обратить внимание на большую "хрупкость" модели: даже наличие/отсутствие финальной точки может сильно повлиять на результат.

In [336]:
for replacement_cand in ["является", "будет", "было", "есть"]:

    ru_text = "На улицах необузданное <nsubj> распространение пессимизма .".replace("<nsubj>", replacement_cand)

    res, _ = translate(
        ru_text[:-1],
        "rus_Cyrl", "rus_Cyrl",
        output_attentions=0, return_dict_in_generate=1,
        bad_words_ids=bad_words_ids
    )
    print(res)


['На улицах бескорыстным является распространение пессимизма']
['На улицах будет бескорыстно распространяться пессимизм']
['На улицах бескорыстно распространялся пессимизм']
['На улицах бескорыстно распространяется пессимизм']


### Стратегии возврата местоимения

Попробуем более точно выделять случаи с пропущенным местоимением. Будем при помощи udpipe находить глаголы, с которыми не связаны слова в именительном падеже (deprel-отношение `nsubj`)
т.е. при помощи)

In [429]:
prodrop_list_filtered = []

for entry in prodrop_list:
    ru_text, cs_text = entry[2].strip(), entry[3].strip()
    doc = nlp_cs(cs_text)
    for token in doc:
        if token.morph and token.pos_ in ["VERB"]:
            nsubj_found = False
            for child in token.children:
                if child.dep_ == "nsubj" and child.head.i == token.i:
                    nsubj_found = True
                    break
            if not nsubj_found:
                prodrop_list_filtered.append(entry)
                break
                
len(prodrop_list_filtered), len(prodrop_list)

(10368, 17962)

In [437]:
CZ_PRONOUNS = {
    ("1", "Sing"): ["já"],
    ("2", "Sing"): ["ty"],
    ("3", "Sing"): ["on", "ona", "ono"],
    ("1", "Plur"): ["my"],
    ("2", "Plur"): ["vy"],
    ("3", "Plur"): ["oni", "one", "ona"]
}

for entry in prodrop_list_filtered[:11]:
    ru_text, cs_text = entry[2].strip(), entry[3].strip()
    doc = nlp_cs(cs_text)
    print(ru_text)
    print(cs_text)
    variants = set()
    for token in doc:
        if token.morph and token.pos_ in ["VERB"]:
            feature_dict = dict(item.split("=") for item in str(token.morph).split("|"))
            if feature_dict.get('VerbForm') == "Inf":
                break

            nsubj_found = False
            for child in token.children:
                if child.dep_ == "nsubj" and child.head.i == token.i:
                    nsubj_found = True
                    break
            if not nsubj_found:
                print()
                print(token.text, token.pos_, token.dep_, token.head)
                print(feature_dict)
                print(list(token.children))
                if "Person" in feature_dict:
                    key = (feature_dict["Person"], feature_dict["Number"])
                    possible_pronouns = CZ_PRONOUNS[key]
                else:
                    possible_pronouns = []
                    for pers in "123":
                        key = (pers, feature_dict["Number"])
                        possible_pronouns += CZ_PRONOUNS[key]
                for one_pronoun in possible_pronouns:
                    cs_text_cand = cs_text.replace(token.text, one_pronoun + " " + token.text)
                    # print(cs_text_cand)
                    res, _ = translate(
                        cs_text_cand,
                        "ces_Latn", "ces_Latn",
                        output_attentions=0, return_dict_in_generate=1,
                        num_beams=8,
                        do_sample=True,  # Enable sampling
                        top_k=50,
                        top_p=0.95,  # Use nucleus sampling
                        temperature=0.9,  # Slight randomness
                        num_return_sequences=8  # Return 5 variants
                    )
                    # print(res)
                    variants |= set(res)
    print(list(variants))


Многие пакистанцы , лишённые иллюзий в отношении политического класса Пакистана , молчаливо согласились с его приходом к власти , думая , что он сможет что -то изменить .
Mnoho Pákistánců rozčarovaných pákistánskou politickou garniturou mlčelo v domnění , že by mohl svým slibům dostát .

mohl VERB acl domnění
{'Gender': 'Masc', 'Number': 'Sing', 'Polarity': 'Pos', 'Tense': 'Past', 'VerbForm': 'Part', 'Voice': 'Act'}
[,, že, by, dostát]
['Mnoho Pákistánců zklamaných pakistánskou politickou garniturou mlčel , že bych mohl splnit své sliby .', 'Mnoho Pákistánců , kteří byli zklamaní pákistánskou politickou garniturou , mlčel , protože si mysleli , že může dosáhnout svých slibů .', 'Mnoho Pákistánců zklamaných pákistánskou politickou garniturou mlčel , že by mohla splnit své sliby .', 'Mnoho Pákistánců , kteří byli zklamaní pakistánskou politickou šatou , mlčel , že by mohl splnit své sliby .', 'Mnoho Pákistánců , kteří byli zklamaní pákistánskou politickou garniturou , mlčel , že by mohl 

KeyError: ('1', 'Plur,Sing')

Детекция пропуска местоименний через udpipe работает неплохо, она правильно находит глагол, у которого нет субъекта.

Добавить местоимение оказывается нелегко: даже с высокой температурой и несколькими кандидатами из beam search, модель очень не хочет генерировать предложения не на "стандартном" чешском. Возможно, стоит попробовать подставлять "фальшивые" имена вместо местоимений, а затем заменять их на правильные местоимения на этапе постпроцессинга.

Что-то похожее наблюдалось и для русского языка. Обычно хорошие результаты получаются либо за счёт парафраза (напр., переделка "Одна альтернатива - это X в "Одна альтернатива заключается в X"), либо за счёт изменения грамматического времени на прошлое или будущее.